In [83]:
# BASIC_SELFACT_CBC.IPYNB - Characterisation of a self-activating gene's RC properties using constitutive reporters, with a basic model
# By Kirill Sechkar

# PACKAGE IMPORTS
import numpy as np
import scipy as sp
import jax
import jax.numpy as jnp
import jaxopt
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
import matplotlib as mpl, matplotlib.pyplot as plt
from bokeh import io as bkio
from bokeh.colors import RGB as bkRGB

# SOLVER AND CONTROLLER IMPORTS
import common.controllers as ctrls
import common.reference_switchers as refsws
import common.ode_solvers as odesols

# BIFURCATION ANALYSIS, PROBE CHARACTERISATION AND PREDICTION TOOLS
from common import selfact_an_bif as an_tools
from common.selfact_jointexp import *
from common.probe_char_tools import *

# CELL AND CIRCUIT MODEL IMPORTS
from basic_model import *
import basic_genetic_modules as gms

# set up jax
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)

# set up bokeh
bkio.reset_output()
bkplot.output_notebook()

# set up matplotlib
%matplotlib widget

Loading BokehJS ...

In [84]:
# INITIALISE THE MODEL, ITS PARAMETERS AND INITIAL CONDITIONS
model_auxil = ModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = model_auxil.default_params()  # get default parameter values
init_conds = model_auxil.default_init_conds(par)  # get default initial conditions

In [85]:
# DEFINE CIRCUIT PARAMETERS TO BE CONSIDERED
circ_par={}

circ_par['q_switch'] = 1.25e2  # resource competition factor for the switch gene
circ_par['q_ofp'] = 100*circ_par['q_switch'] # RC factor for the switch's fluorescent output gene
circ_par['baseline_switch'] = 0.05
circ_par['K_switch'] = 250.0
circ_par['I_switch'] = 0.1

# default RC factor for a constitutive reporter
circ_par['q_ofp2'] = 6e4

# range of RC factors for the constitutive reporters
constrep_qs = np.flip(np.linspace(45.0,4e4,10))
# define maturation rates considered
constrep_mus = [1/(13.6/60)]*len(constrep_qs)  # sfGFP maturation time of 13.6 min

In [86]:
# INITIALISE AND PARAMETERISE THE CELL MODEL

# initialise cell model
model_auxil = ModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
model_par = model_auxil.default_params()  # get default parameter values
init_conds = model_auxil.default_init_conds(model_par)  # get default initial conditions

# add reference tracker switcher
model_par_with_refswitch, ref_switcher = model_auxil.add_reference_switcher(model_par,
                                                                                    # cell model parameters
                                                                                    refsws.no_switching_initialise,
                                                                                    # function initialising the reference switcher
                                                                                    refsws.no_switching_switch
                                                                                    # function switching the references to be tracked
                                                                                    )

# load synthetic genetic modules and the controller
odeuus_complete, \
    module1_F_calc, module2_F_calc, \
    module1_specterms, module2_specterms, \
    controller_action, controller_update, \
    par, init_conds, controller_memo0, \
    synth_genes_total_and_each, synth_miscs_total_and_each, \
    controller_memos, controller_dynvars, controller_ctrledvar, \
    modules_name2pos, modules_styles, controller_name2pos, controller_styles, \
    module1_v_with_F_calc, module2_v_with_F_calc = model_auxil.add_modules_and_controller(
        # module 1
        gms.sas_initialise,  # function initialising the circuit
        gms.sas_ode,  # function defining the circuit ODEs
        gms.sas_F_calc, # function calculating the circuit genes' transcription regulation functions
        gms.sas_specterms, # function calculating the circuit genes effective mRNA levels (due to possible co-expression from the same operons)
        # module 2
        gms.constfp2_initialise,  # function initialising the circuit
        gms.constfp2_ode,  # function defining the circuit ODEs
        gms.constfp2_F_calc, # function calculating the circuit genes' transcription regulation functions
        gms.constfp2_specterms, # function calculating the circuit genes effective mRNA levels (due to possible co-expression from the same operons)
        # controller
        ctrls.cci_initialise,  # function initialising the controller
        ctrls.cci_action,  # function calculating the controller action
        ctrls.cci_ode,  # function defining the controller ODEs
        ctrls.cci_update,  # function updating the controller based on measurements
        # cell model parameters and initial conditions
        model_par_with_refswitch, init_conds)

# unpack the synthetic genes and miscellaneous species lists
synth_genes = synth_genes_total_and_each[0]
module1_genes = synth_genes_total_and_each[1]
module2_genes = synth_genes_total_and_each[2]
synth_miscs = synth_miscs_total_and_each[0]
module1_miscs = synth_miscs_total_and_each[1]
module2_miscs = synth_miscs_total_and_each[2]

# update circuit parameters
par.update(circ_par)

In [87]:
# GET MAXIMUM BURDENS AND DEGRADATION COEFFICIENTS

q_r_and_q_o = par['q_r'] + par['q_o']

# calculate maximum synthetic gene burdens
Qmaxs = {
    'Q_switch_max': par['q_switch'] / q_r_and_q_o,
    'Q_ofp_max': par['q_ofp'] / q_r_and_q_o,
}

# calculate native gene burdens
Qs_native = {
    'Q_r': par['q_r'] / q_r_and_q_o,
    'Q_o': par['q_o'] / q_r_and_q_o,
}

# no degradation => zero chis
chis={'chi_switch':0.0, 'chi_ofp':0.0}

# record maximum burdens and degradation coefficients in a common dictionary
cellvars={}
cellvars.update(Qmaxs)
cellvars.update(Qs_native)
cellvars.update(chis)
# add maxmium overall burden from the self-activating switch
cellvars['Q_sas_max']=cellvars['Q_switch_max']+cellvars['Q_ofp_max']
# add translation elongation rate
cellvars['e']=par['e']

# set zero 'housekeeping protein mass fraction' parameter, just to reuse the cell model tools
par['phi_q'] = 0.0

In [88]:
# GET THE TWO BIFURCATION POINTS

# for one of them, F_req touches the F_real curve from below; for the other, from above
bif_Freqbelow, bif_Freqabove = an_tools.find_bifurcations(par, cellvars)

In [89]:
# GET THE ANALYTICAL BIFURCATION CURVE

# define the range of probe-exerted burdens to consider
Q_probe_range=np.linspace(0.0, 0.5, 20)

# get the bifurcation curve
an_bif_curve = an_tools.find_equilibria_for_Q_osynth_range(Q_probe_range,
                                                           bif_Freqbelow, bif_Freqabove,
                                                           par, cellvars)

In [90]:
# PLOT THE ANALYTICAL BIFURCATION CURVE

# p_ofp vs imposed burden
an_ofp_fig = bkplot.figure(
    frame_width=360,
    frame_height=360,
    x_axis_label="Q_p, probe burden",
    y_axis_label="ofp_mature, mature output fluorescent protein level",
    x_range=(min(Q_probe_range), max(Q_probe_range)),
    y_range=(0, max(an_bif_curve['ofp_mature'])),
    title='Analytical bifurcation curve: p_ofp vs imposed burden',
    tools="box_zoom,pan,hover,reset"
)
# plot the controlled variable vs control action
an_ofp_fig.line(x=an_bif_curve['Q_osynth'],
             y=an_bif_curve['ofp_mature'],
             line_width=1.5, line_color='black', line_dash='dashed',
             legend_label='true steady states')
an_ofp_fig.scatter(x=an_bif_curve['Q_osynth'],
                y=an_bif_curve['ofp_mature'],
                marker='circle', size=5,
                color='black', legend_label='true steady states')
# legend formatting
an_ofp_fig.legend.label_text_font_size = "8pt"
an_ofp_fig.legend.location = "top_right"
an_ofp_fig.legend.click_policy = 'hide'

# exreted burden vs imposed burden
an_Q_fig = bkplot.figure(
    frame_width=360,
    frame_height=360,
    x_axis_label="Q_p, probe burden",
    y_axis_label="Q_sas, self-activating switch burden",
    x_range=(0, 0.5),
    y_range=(0, 0.5),
    title='Analytical bifurcation curve: exerted vs imposed burden',
    tools="box_zoom,pan,hover,reset"
)
# plot the controlled variable vs control action
an_Q_fig.line(x=an_bif_curve['Q_osynth'],
             y=an_bif_curve['Q_sas'],
             line_width=1.5, line_color='black', line_dash='dashed',
             legend_label='true steady states')
an_Q_fig.scatter(x=an_bif_curve['Q_osynth'],
                y=an_bif_curve['Q_sas'],
                marker='circle', size=5,
                color='black', legend_label='true steady states')
# legend formatting
an_Q_fig.legend.label_text_font_size = "8pt"
an_Q_fig.legend.location = "top_right"
an_Q_fig.legend.click_policy = 'hide'

# show the plots
bkplot.show(bklayouts.grid([[an_ofp_fig, an_Q_fig]]))

In [91]:
# SIMULATE MEASUREMENTS - SET CONTROLLER AND SIMULATION PARAMETER

# controller parameters irrelevant, as we just have a constant zero chemical input
controller_ctrledvar = 'ofp_mature'  # controlled variable
control_delay = 0.0
ref = jnp.array([0.0])
u0 = 0.0

# simulation parameters
tf = (0.0, 72.0)  # assuming steady state reached in 24h
meastimestep = 0.1  # hours
euler_timestep = 1e-5

# create an ODE solver
ode_solver, us_size = odesols.create_euler_solver(odeuus_complete,
                                                  control_delay=control_delay,
                                                  meastimestep=meastimestep,
                                                  euler_timestep=euler_timestep)

In [92]:
# SIMULATE MEASUREMENTS FOR REPORTERS ALONE FIRST
# save q_swith for after this simulation, where it will be set to zero
q_switch_bckup = par['q_switch']
q_ofp_bckup = par['q_ofp']
mature_ofp2_ss_individual = np.zeros(len(constrep_qs))  # initialise storage for steady-state mature protein levels
l_ss_individual = np.zeros(len(constrep_qs))  # initialise storage for steady-state cell growth rates
for i in range(0, len(constrep_qs)):
    # set constitutive reporter parameters depending on which reporter is considered
    par['q_switch'] = 0.0
    par['q_ofp'] = 0.0
    par['q_ofp2'] = constrep_qs[i]
    par['mu_ofp2'] = constrep_mus[i]

    # solve ODE
    ts_jnp, xs_jnp, \
        ctrl_ss_memorecord_jnp, uexprecord_jnp, \
        refrecord_jnp = ode_sim(par,  # model parameters
                                ode_solver,  # ODE solver for the cell with the synthetic gene circuit
                                odeuus_complete,
                                # ODE function for the cell with the synthetic gene circuit and the controller (also gives calculated and experienced control actions)
                                controller_ctrledvar,  # name of the variable read and steered by the controller
                                controller_update, controller_action,
                                # function for updating the controller memory and calculating the control action
                                model_auxil.x0_from_init_conds(init_conds,
                                                               par,
                                                               synth_genes, synth_miscs, controller_dynvars,
                                                               modules_name2pos,
                                                               controller_name2pos),  # initial condition VECTOR
                                controller_memo0,  # initial controller memory record
                                u0,
                                # initial control action, applied before any measurement-informed actions reach the sy_probestem
                                (len(synth_genes), len(module1_genes), len(module2_genes)),  # number of synthetic genes
                                (len(synth_miscs), len(module1_miscs), len(module2_miscs)),
                                # number of miscellaneous species
                                modules_name2pos, controller_name2pos,
                                # dictionaries mapping gene names to their positions in the state vector
                                model_auxil.synth_gene_params_for_jax(par, synth_genes),
                                # synthetic gene parameters in jax.array form
                                tf, meastimestep,  # simulation time frame and measurement time step
                                control_delay,  # delay before control action reaches the sy_probestem
                                us_size,  # size of the control action record needed
                                ref, ref_switcher,  # reference values and reference switcher
                                )
    # convert simulation results to numpy array_probes
    ts = np.array(ts_jnp)
    xs = np.array(xs_jnp)
    uexprecord = np.array(uexprecord_jnp)

    # record mature protein levels
    mature_ofp2_ss_individual[i] = xs[-1, modules_name2pos['ofp2_mature']]

    # calculate and record growth rates
    _, ls_jnp, _, _, _, _, _ = model_auxil.get_e_l_Fr_nu_psi_T_D(ts, xs, par,
                                                                 synth_genes, synth_miscs,
                                                                 modules_name2pos,
                                                                 module1_specterms, module2_specterms,
                                                                 # arguments only used by the basic model
                                                                 module1_F_calc, module2_F_calc,
                                                                 uexprecord,
                                                                 synth_genes_total_and_each, synth_miscs_total_and_each
                                                                 )
    l_ss_individual[i] = np.array(ls_jnp)[-1]

# set switch and ofp parameters back to their original values
par['q_switch'] = q_switch_bckup
par['q_ofp'] = q_ofp_bckup

In [93]:
# SIMULATE MEASUREMENTS WITH REPORTERS - RUN THE SIMULATION
mature_ofp_ss = np.zeros(len(constrep_qs))
F_switch_ss = np.zeros(len(constrep_qs))
mature_ofp2_ss_withsas = np.zeros(len(constrep_qs))  # initialise storage for steady-state mature protein levels
l_ss_withsas = np.zeros(len(constrep_qs))  # initialise storage for steady-state cell growth rates
for i in range(0, len(constrep_qs)):
    # set constitutive reporter parameters depending on which reporter is considered
    par['q_ofp2'] = constrep_qs[i]
    par['mu_ofp2'] = constrep_mus[i]
    
    # set up the ODE solver
    ode_solver, us_size = odesols.create_euler_solver(odeuus_complete,
                                                      control_delay=control_delay,
                                                      meastimestep=meastimestep,
                                                      euler_timestep=1e-5)

    # solve ODE
    timer = time.time()
    ts_jnp, xs_jnp, \
        ctrl_memorecord_jnp, uexprecord_jnp, \
        refrecord_jnp = ode_sim(par,  # model parameters
                                ode_solver,  # ODE solver for the cell with the synthetic gene circuit
                                odeuus_complete,
                                # ODE function for the cell with the synthetic gene circuit and the controller (also gives calculated and experienced control actions)
                                controller_ctrledvar,  # name of the variable read and steered by the controller
                                controller_update, controller_action,
                                # function for updating the controller memory and calculating the control action
                                model_auxil.x0_from_init_conds(init_conds,
                                                               par,
                                                               synth_genes, synth_miscs, controller_dynvars,
                                                               modules_name2pos,
                                                               controller_name2pos),  # initial condition VECTOR
                                controller_memo0,  # initial controller memory record
                                u0,
                                # initial control action, applied before any measurement-informed actions reach the system
                                (len(synth_genes), len(module1_genes), len(module2_genes)),  # number of synthetic genes
                                (len(synth_miscs), len(module1_miscs), len(module2_miscs)),
                                # number of miscellaneous species
                                modules_name2pos, controller_name2pos,
                                # dictionaries mapping gene names to their positions in the state vector
                                model_auxil.synth_gene_params_for_jax(par, synth_genes),
                                # synthetic gene parameters in jax.array form
                                tf, meastimestep,  # simulation time frame and measurement time step
                                control_delay,  # delay before control action reaches the system
                                us_size,  # size of the control action record needed
                                ref, ref_switcher,  # reference values and reference switcher
                                )

    # convert simulation results to numpy arrays
    ts = np.array(ts_jnp)
    xs = np.array(xs_jnp)
    ctrl_memorecord = np.array(ctrl_memorecord_jnp)
    uexprecord = np.array(uexprecord_jnp)
    refrecord = np.array(refrecord_jnp)

    # record mature protein levels
    mature_ofp_ss[i] = xs[-1, modules_name2pos['ofp_mature']]
    mature_ofp2_ss_withsas[i] = xs[-1,modules_name2pos['ofp2_mature']]

    # calculate and record growth rates
    _, ls_jnp, _, _, _, _, _ = model_auxil.get_e_l_Fr_nu_psi_T_D(ts, xs, par,
                                                                 synth_genes, synth_miscs,
                                                                 modules_name2pos,
                                                                 module1_specterms, module2_specterms,
                                                                 # arguments only used by the basic model
                                                                 module1_F_calc, module2_F_calc,
                                                                 uexprecord,
                                                                 synth_genes_total_and_each, synth_miscs_total_and_each
                                                                 )
    l_ss_withsas[i] = np.array(ls_jnp)[-1]

    # calculate the switch gene's transcription regulation function
    F_switch_ss[i] = module1_F_calc(ts[-1], xs[-1, :], uexprecord[-1],
                                    par=par, name2pos=modules_name2pos)[0]

In [94]:
# GET THE NORMALISED BURDENS OF CONSTITUTIVE REPORTERS
Q_constrep_steady_states=constrep_qs/(par['q_r']+par['q_o'])

In [95]:
# GET THE BURDENS IMPOSED BY THE SELF-ACTIVATING SWITCH
Q_sas_steady_states=np.zeros_like(constrep_qs)
Q_sas_steady_states_est=np.zeros_like(constrep_qs)
for i in range(0, len(constrep_qs)):
    Q_sas_steady_states[i] = F_switch_ss[i] * (par['q_switch']+par['q_ofp'])/(par['q_r']+par['q_o'])
    # estimated from
    Q_sas_steady_states_est[i] = Q_calc_one(constrep_mus[i],  # maturation rate of the constitutive reporter's output fluorescent protein
                           mature_ofp2_ss_individual[i], l_ss_individual[i],    # individual measurements for the const. reporter: output prot. conc., growth rate
                           mature_ofp2_ss_withsas[i], l_ss_withsas[i],  # pair measurements for the const. reporter: output prot. concs., growth rate
                           Q_constrep_steady_states[i]    # normalised resource competition factor for the constitutive reporter
                           )  # using j-1 for ofpp level and growth rate as the constitutive reporter list has an extra zero at the beginning

In [96]:
# PLOT BIFURCATION DIAGRAM

# plot the steady states as actually observed
constrep_fig = bkplot.figure(
    frame_width=480,
    frame_height=360,
    x_axis_label="Q_constrep",
    y_axis_label="ofp_mature",
    # x_range=(0.0, 0.5),
    # y_range=(0, max(an_bif_curve['ofp_mature'])),
    tools="box_zoom,pan,hover,reset,save"
)
constrep_fig.output_backend = 'svg'

# plot the steady states
constrep_fig.scatter(x=Q_constrep_steady_states, y=mature_ofp_ss,
                marker='circle', size=7.5, 
               color='violet', legend_label='steady states')

# legend formatting
constrep_fig.legend.label_text_font_size = "8pt"
constrep_fig.legend.location = "bottom_left"
constrep_fig.legend.click_policy = 'hide'

# plot the cbc trajectory for imposed and exerted burdens
constrep_Q_fig = bkplot.figure(
    frame_width=480,
    frame_height=360,
    x_axis_label="Q_constrep, reporter resource demand",
    y_axis_label="Q_sas, self-activating switch's demand",
    x_range=(0, 0.75),
    y_range=(0, 0.25),
    title='Burden plot',
    tools="box_zoom,pan,hover,reset,save"
)
constrep_Q_fig.output_backend = 'svg'
# plot the analytical bifurcation curve
constrep_Q_fig.line(x=an_bif_curve['Q_osynth'],
             y=an_bif_curve['Q_sas'],
             line_width=1.5, line_color='black', line_dash='dashed',
             legend_label='true steady states')
constrep_Q_fig.scatter(x=an_bif_curve['Q_osynth'],
                y=an_bif_curve['Q_sas'],
                marker='circle', size=5,
                color='black', legend_label='true steady states')

# plot the steady states
constrep_Q_fig.scatter(x=Q_constrep_steady_states, y=Q_sas_steady_states,
                marker='circle', size=7.5, 
               color='violet', legend_label='observed steady states (real)')
# plot the estimated steady states
constrep_Q_fig.scatter(x=Q_constrep_steady_states, y=Q_sas_steady_states_est,
                marker='circle', size=7.5,
               color='darkviolet', legend_label='observed steady states (est)')

# legend formatting
constrep_Q_fig.legend.label_text_font_size = "8pt"
constrep_Q_fig.legend.location = "top_right"
constrep_Q_fig.legend.click_policy = 'hide'


# show plot
bkplot.show(bklayouts.grid([[constrep_fig, constrep_Q_fig]]))

In [97]:
# CDC PAPER - FIGURE 1 A
# plot the steady states as actually observed
intro_a_fig = bkplot.figure(
    frame_width=100,
    frame_height=100,
    x_axis_label="Resource demand\nfrom CRi",
    y_axis_label="MOI output",
    x_range=(0.0, 0.6),
    y_range=(0, 5e5),
    tools="box_zoom,pan,hover,reset,save"
)
intro_a_fig.output_backend = 'svg'

# plot the possible steady states
intro_a_fig.line(x=an_bif_curve['Q_osynth'], y=an_bif_curve['ofp_mature'],
                 line_width=1.5, line_dash='dashed', color='black',
                 legend_label='Equilibria')

# plot the characterisation outcomes
intro_a_fig.scatter(x=Q_constrep_steady_states, y=mature_ofp_ss,
                    marker='circle', size=5, color='#bb3385ff',
                    legend_label='Observations')

# legend formatting
intro_a_fig.legend.location = "top_right"
intro_a_fig.legend.label_text_font_size = "7pt"
intro_a_fig.legend.padding=0
intro_a_fig.legend.margin=2
intro_a_fig.legend.spacing=0
intro_a_fig.legend.glyph_width=5
intro_a_fig.legend.click_policy = 'hide'

# axis formatting
intro_a_fig.yaxis.formatter=bkmodels.PrintfTickFormatter(format="%4.0e")
intro_a_fig.yaxis.ticker=bkmodels.BasicTicker(desired_num_ticks=6)
intro_a_fig.xaxis.axis_label_text_font_size = "8pt"
intro_a_fig.yaxis.axis_label_text_font_size = "8pt"
intro_a_fig.xaxis.axis_label_text_color = "black"
intro_a_fig.yaxis.axis_label_text_color = "black"
intro_a_fig.xaxis.major_label_text_font_size = "7pt"
intro_a_fig.yaxis.major_label_text_font_size = "7pt"
intro_a_fig.xaxis.major_label_text_color = "black"
intro_a_fig.yaxis.major_label_text_color = "black"

bkplot.show(intro_a_fig)

In [110]:
# PARAMETER IDENTIFICATION BASED ON MEASUREMENTS

perfectly_fitted_parvec = np.array([
    cellvars['Q_switch_max'],
    cellvars['Q_ofp_max'],
    par['n_switch'],
    par['K_switch'],
    par['I_switch'],
    par['baseline_switch'],
    par['eta_switch']
])

init_parvec = jnp.array([
    0.25,
    0.25,
    200.0,
    0.5,
    500.0,
    0.5,
    1.0
])

diffevo_bounds=[]
for i in range(0,len(perfectly_fitted_parvec)):
    diffevo_bounds.append((perfectly_fitted_parvec[i]/100, perfectly_fitted_parvec[i]*100))

bounds=(perfectly_fitted_parvec/10, perfectly_fitted_parvec*10)



In [111]:
cost_func = lambda fitted_parvec: an_tools.id_cost(Q_sas_steady_states_est, Q_constrep_steady_states,
                                                   par, cellvars,
                                                   fitted_parvec)
curve_func = lambda Q_sas_range, Q_switch_max, Q_ofp_max, n_switch, K_switch, I_switch, baseline_switch, eta_switch: an_tools.find_equilibria_for_Q_sas_range_id(Q_sas_range,
                                                                   # range of burdens from other genes to consider
                                                                   par,
                                                                   Q_switch_max,
                                                                   Q_ofp_max,
                                                                   n_switch,
                                                                   K_switch,
                                                                   I_switch,
                                                                   baseline_switch,
                                                                   eta_switch
                                                                   )

In [112]:
# res = sp.optimize.curve_fit(curve_func, Q_sas_steady_states_est, Q_constrep_steady_states,
#                                    p0=perfectly_fitted_parvec*1.05, maxfev=100000,
#                                    nan_policy='omit', method='dogbox', bounds=bounds)
# fitted_parvec = res[0]
res = sp.optimize.differential_evolution(cost_func,bounds=diffevo_bounds)
fitted_parvec = res.x
# print('Fitted parameters: '+str(fitted_parvec))
# print('Fitted parameter cost: '+str(cost_func(fitted_parvec)))

In [113]:
# PLOT the fit
fit_fig = bkplot.figure(
    frame_width=480,
    frame_height=360,
    x_axis_label="Q_constrep, reporter resource demand",
    y_axis_label="Q_sas, self-activating switch's demand",
    # x_range=(0, 0.75),
    # y_range=(0, 0.25),
    title='Burden plot',
    tools="box_zoom,pan,hover,reset,save"
)
fit_fig.output_backend = 'svg'
# plot the analytical bifurcation curve
fit_fig.line(x=an_bif_curve['Q_osynth'],
             y=an_bif_curve['Q_sas'],
             line_width=1.5, line_color='black', line_dash='dashed',
             legend_label='true steady states')
fit_fig.scatter(x=an_bif_curve['Q_osynth'],
                y=an_bif_curve['Q_sas'],
                marker='circle', size=5,
                color='black', legend_label='true steady states')

# plot the estimated steady states
fit_fig.scatter(x=Q_constrep_steady_states, y=Q_sas_steady_states_est,
                marker='circle', size=7.5,
               color='darkviolet', legend_label='observed steady states (est)')

# legend formatting
fit_fig.legend.label_text_font_size = "8pt"
fit_fig.legend.location = "top_right"
fit_fig.legend.click_policy = 'hide'


# plot the fitted bifrucation curve
Q_sas_mesh=np.linspace(min(Q_sas_steady_states_est),max(Q_sas_steady_states_est),100)
Q_constrep_fit = an_tools.find_equilibria_for_Q_sas_range_id(Q_sas_mesh, par,
                                                             fitted_parvec[0], fitted_parvec[1], fitted_parvec[2], fitted_parvec[3],
                                                             fitted_parvec[4], fitted_parvec[5], fitted_parvec[6])
fit_fig.line(x=Q_constrep_fit, y=Q_sas_mesh,
             line_width=2.0,
               color='cyan', legend_label='fitting')

bkplot.show(fit_fig)